In [3]:
import os
import torch
import torchaudio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Configuration
CONFIG = {'SEED': 42, 'SR': 32000}

# Load data
df = pd.read_csv('./train.csv')
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG['SEED'])
test = pd.read_csv('./test.csv')
train.to_csv('train_a.csv', index=False)
val.to_csv('val_a.csv', index=False)

def save_spectrogram_image(y, sr, out_path, n_mels=128, hop_length=256):
    y = torch.tensor(y).cuda()
    S = torchaudio.transforms.MelSpectrogram(sample_rate=sr, n_mels=n_mels, hop_length=hop_length).cuda()
    S_dB = torchaudio.transforms.AmplitudeToDB()(S(y))
    plt.figure(figsize=(10, 4))
    plt.imshow(S_dB.cpu().numpy(), aspect='auto', origin='lower')
    plt.axis('off')
    plt.savefig(out_path, bbox_inches='tight', pad_inches=0, format='png')
    plt.close()

def convert_audio_to_spectrogram_batch(df, output_folder, batch_size=100):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    num_batches = len(df) // batch_size + 1
    for batch_num in range(num_batches):
        batch_df = df.iloc[batch_num * batch_size : (batch_num + 1) * batch_size]
        for _, row in tqdm(batch_df.iterrows(), total=len(batch_df)):
            y, sr = torchaudio.load(row['path'])
            y = y.numpy().squeeze()
            file_name = os.path.basename(row['path']).replace('.ogg', '.png')
            out_path = os.path.join(output_folder, file_name)
            save_spectrogram_image(y, sr, out_path)

# Convert audio to spectrograms
#convert_audio_to_spectrogram_batch(train, './train_spectrograms/')
#convert_audio_to_spectrogram_batch(val, './val_spectrograms/')
#convert_audio_to_spectrogram_batch(test, './test_spectrograms/')


In [4]:
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image

class SpectrogramDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_map = {'real': 1, 'fake': 0}  # Add a mapping for labels

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0].replace('.ogg', '.png'))+".png"
        image = Image.open(img_name).convert('RGB')
        label = self.data_frame.iloc[idx, 2]
        
        if self.transform:
            image = self.transform(image)
        
        label = self.label_map[label]  # Map the string label to an integer
        label = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        
        return image, label
class TestSpectrogramDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx, 1].replace('./test','./test_spectrograms').replace('.ogg', '.png')
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return image

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Datasets
train_dataset = SpectrogramDataset(csv_file='./train_a.csv', root_dir='./train_spectrograms/', transform=transform)
val_dataset = SpectrogramDataset(csv_file='./val_a.csv', root_dir='./val_spectrograms/', transform=transform)
test_dataset = TestSpectrogramDataset(csv_file='./test.csv', transform=transform)



In [5]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torchvision.models import vision_transformer, ViT_B_16_Weights, ViT_B_32_Weights, ViT_L_16_Weights, ViT_L_32_Weights, ViT_H_14_Weights
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

def train_model(model_name, weights, num_classes=2, num_epochs=10, batch_size=32, learning_rate=0.001):
    # Initialize the model
    model = getattr(vision_transformer, model_name)(weights=weights)
    num_ftrs = model.heads.head.in_features
    model.heads.head = nn.Linear(num_ftrs, num_classes)  # Adjust output layer for binary classification
    
    # Move model to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Assuming train_dataset, val_dataset, and test_dataset are already defined
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training Loop with Logging
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        
        # Validation Loop with Logging
        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)
        
        val_loss = val_running_loss / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}')

    return model, train_loader, val_loader, test_loader


In [4]:
trained_model, train_loader, val_loader, test_loader = train_model('vit_b_32', ViT_B_32_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/vit_b_32-d86f8d99.pth" to C:\Users\p/.cache\torch\hub\checkpoints\vit_b_32-d86f8d99.pth
100%|███████████████████████████████████████████████████████████████████████████████| 337M/337M [00:58<00:00, 6.01MB/s]
Epoch 1/10 Training:   0%|                                                                    | 0/1386 [00:00<?, ?it/s]C:\Users\p\anaconda3\envs\ws\Lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Epoch 1/10 Training: 100%|█████████████████████████████████████████████████████████| 1386/1386 [07:02<00:00,  3.28it/s]


Epoch 1/10, Loss: 0.4101


Epoch 1/10 Validation: 100%|█████████████████████████████████████████████████████████| 347/347 [01:40<00:00,  3.46it/s]


Validation Loss: 0.1463


Epoch 2/10 Training: 100%|█████████████████████████████████████████████████████████| 1386/1386 [04:30<00:00,  5.12it/s]


...

Validation Loss: 0.0196


Epoch 10/10 Training: 100%|████████████████████████████████████████████████████████| 1386/1386 [04:22<00:00,  5.28it/s]


Epoch 10/10, Loss: 0.0211


Epoch 10/10 Validation: 100%|████████████████████████████████████████████████████████| 347/347 [01:00<00:00,  5.69it/s]

Validation Loss: 0.0423


In [4]:
trained_model, train_loader, val_loader, test_loader = train_model('vit_l_16', ViT_L_16_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/vit_l_16-852ce7e3.pth" to C:\Users\p/.cache\torch\hub\checkpoints\vit_l_16-852ce7e3.pth
100%|█████████████████████████████████████████████████████████████████████████████| 1.13G/1.13G [02:00<00:00, 10.1MB/s]
Epoch 1/10 Training:   0%|                                                                    | 0/1386 [00:00<?, ?it/s]C:\Users\p\anaconda3\envs\ws\Lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Epoch 1/10 Training: 100%|███████████████████████████████████████████████████████| 1386/1386 [1:04:18<00:00,  2.78s/it]


Epoch 1/10, Loss: 0.7137


Epoch 1/10 Validation: 100%|█████████████████████████████████████████████████████████| 347/347 [13:45<00:00,  2.38s/it]


Validation Loss: 0.6931


Epoch 2/10 Training: 100%|███████████████████████████████████████████████████████| 1386/1386 [1:04:22<00:00,  2.79s/it]


...

Validation Loss: 0.2124


Epoch 10/10 Training: 100%|██████████████████████████████████████████████████████| 1386/1386 [1:04:17<00:00,  2.78s/it]


Epoch 10/10, Loss: 0.1949


Epoch 10/10 Validation: 100%|████████████████████████████████████████████████████████| 347/347 [13:48<00:00,  2.39s/it]

Validation Loss: 0.1761


In [4]:
trained_model, train_loader, val_loader, test_loader = train_model('vit_l_32', ViT_L_32_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/vit_l_32-c7638314.pth" to C:\Users\p/.cache\torch\hub\checkpoints\vit_l_32-c7638314.pth
100%|██████████| 1.14G/1.14G [01:45<00:00, 11.7MB/s]
Epoch 1/10 Training:   0%|          | 0/1386 [00:00<?, ?it/s]C:\Users\p\anaconda3\envs\ws\Lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Epoch 1/10 Training: 100%|██████████| 1386/1386 [08:04<00:00,  2.86it/s]


Epoch 1/10, Loss: 0.4963


Epoch 1/10 Validation: 100%|██████████| 347/347 [01:33<00:00,  3.73it/s]


Validation Loss: 0.2427


Epoch 2/10 Training: 100%|██████████| 1386/1386 [06:33<00:00,  3.53it/s]


...

Validation Loss: 0.0377


Epoch 10/10 Training: 100%|██████████| 1386/1386 [06:37<00:00,  3.49it/s]


Epoch 10/10, Loss: 0.0229


Epoch 10/10 Validation: 100%|██████████| 347/347 [01:09<00:00,  4.99it/s]

Validation Loss: 0.0324


In [4]:
trained_model, train_loader, val_loader, test_loader = train_model('vit_h_14', ViT_H_14_Weights.IMAGENET1K_SWAG_LINEAR_V1)

Epoch 1/10 Training:   0%|          | 0/1386 [00:00<?, ?it/s]C:\Users\p\anaconda3\envs\ws\Lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Epoch 1/10 Training:   0%|          | 0/1386 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 122.00 MiB. GPU 

In [5]:
# Save the model
torch.save(trained_model.state_dict(), 'vit_l_32_model.pth')
print('Model saved successfully.')

Model saved successfully.


In [6]:
class Test2SpectrogramDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx, 1].replace('./train','./val_spectrograms').replace('.ogg', '.png')
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return image
test2_dataset = Test2SpectrogramDataset(csv_file='./val_a.csv', transform=transform)
test2_loader = DataLoader(test2_dataset, batch_size=32, shuffle=False, num_workers=0)

In [7]:
import torch
from torchvision.models import vision_transformer

# Define the model architecture
model_name = 'vit_l_32'
weights = ViT_L_16_Weights.IMAGENET1K_V1
model = vision_transformer.vit_l_16(weights=weights)

# Replace the classification head (assuming binary classification)
num_ftrs = model.heads.head.in_features
model.heads.head = torch.nn.Linear(num_ftrs, 2)  # Adjust as necessary for your task

# Load the saved model weights
model.load_state_dict(torch.load('vit_l_16_model.pth'))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the model to evaluation mode
model.eval()

print('Model loaded successfully.')


Model loaded successfully.


In [2]:
train_dataset = SpectrogramDataset(csv_file='./train.csv', root_dir='./train_spectrograms/', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

NameError: name 'SpectrogramDataset' is not defined

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Inference Function with Logging
def inference(model, data_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs in tqdm(data_loader, desc="Inference"):
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            predictions.extend(probabilities.cpu().numpy())
    return predictions

# Inference and Submission
test = pd.read_csv('./test.csv')  # Load your test CSV file
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)
preds = inference(model, test_loader, device)
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = preds
submit.head()

Inference: 100%|██████████| 1563/1563 [09:18<00:00,  2.80it/s]
C:\Users\p\AppData\Local\Temp\ipykernel_19992\1577524310.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.8684624433517456, 0.033276915550231934, 0.015814999118447304, 0.00486947363242507, 0.008451886475086212, 0.06317521631717682, 0.41598081588745117, 0.1489495486021042, 0.0022708189208060503, 0.7465057969093323, 0.7115843296051025, 0.9895880222320557, 0.2727644145488739, 0.015049469657242298, 0.25417986512184143, 0.1338319033384323, 0.034493882209062576, 0.00012611510464921594, 0.008743858896195889, 0.6005038619041443, 0.015298837795853615, 0.036278676241636276, 0.0007480272324755788, 0.7891567945480347, 0.04826914891600609, 0.8822823166847229, 0.008043636567890644, 0.0033115739934146404, 0.34389346837997437, 0.7742266058921814, 0.007796979509294033, 0.005865424871444702, 0.0034826381597667933, 0.0008047191076911986, 0.5115207433700562, 0.90039

,id,fake,real
0,TEST_00000,0.868462,0.131538
1,TEST_00001,0.033277,0.966723
2,TEST_00002,0.015815,0.984185
3,TEST_00003,0.004869,0.995131
4,TEST_00004,0.008452,0.991548


In [24]:
submit.to_csv("test.csv",index=False)